In [1]:
# !pip install geopandas --upgrade

In [2]:
import requests
import geojson
import folium
import geopandas
import psycopg2
import psycopg2.extras
import numpy as np
import itertools
import pandas as pd
import rasterio
import rasterio.mask
import gdal
# import rasterstats

from scipy.stats import pareto
from shapely.ops import cascaded_union
from sqlalchemy import create_engine
from rasterstats import zonal_stats
from shapely import wkt

In [3]:
## Set up access controlls
#Todo: should this be some kind of 'secret' or be stored in a secrets thing?
#Todo: check database to see if a parcel has been queried recently or if we have current enough data
# and returned that analysis instead of buying another parcel and analyisin
db_cred = {'user': 'jifesypi',
            'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
            'host': 'mild-williams-pear.db.elephantsql.com',
            'port': '5432', 
            'database': 'jifesypi',
            'epsg': '4326',
            'schema':'schema'
            }

In [4]:
sonoma_parcel_dataframe = geopandas.read_file('/work/sonoma.gpkg')

In [5]:
parcel_dataframe= sonoma_parcel_dataframe.iloc[[0]]

In [6]:
# Query postgres database based on parcel geometry
#Set query EPSG (WSG 4326)
parcel_dataframe = parcel_dataframe.set_crs('EPSG:4326')

#Convert to 26910, stateplane & meters
parcel_dataframe_26910 = parcel_dataframe.to_crs(26910)

#Buffer parcel 1000 meters
buffer_parcel_dataframe_26910 = parcel_dataframe_26910.buffer(1000).to_crs(4326)

#Extract well known text
parcel_geometry_wkt = buffer_parcel_dataframe_26910.geometry.to_wkt().to_list()[0]

#Create connection to the database
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

#Query for neighborhood of parcels
sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{parcel_geometry_wkt}', 0))"

#Send to engine
neighborhood_parcels = geopandas.read_postgis(sql_query_parcels, engine)

#Need to union to send to wkt query
# neighborhood_parcels_union = geopandas.GeoSeries(unary_union(neighborhood_parcels.geometry.values))
neighborhood_parcels_union = neighborhood_parcels.unary_union

#Get wkt of neighborhood of parcels
neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

#Query for extended neighborhood (1000m) of structures
sql_query_parcels = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

neighborhood_structures = geopandas.read_postgis(sql_query_parcels, engine)

#Query for parcel structures
sql_query_structures = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{parcel_geometry_wkt}',0))"

parcel_structures = geopandas.read_postgis(sql_query_structures, engine)

In [7]:
vegetation_dt = geopandas.read_file('/work/veg_source_dt.csv')
structure_dt = geopandas.read_file('/work/str_source_dt.csv')
assembly_dt = geopandas.read_file('/work/asm_source_dt.csv')

In [8]:
veg_pareto_dt = pd.DataFrame(columns=['Wind_Speed', 'Subs','vec_len', 'b','loc','scale'])
for j, k in itertools.product(['Idle','Medium','High'],['Grass','Shrub','Tree']):
    pareto_vec = vegetation_dt[(vegetation_dt['Vegetation Type']==k) & (vegetation_dt['Wind Speed'] ==j)]['Total Flying Distance  [m]'].values
    pareto_vec = [float(i) for i in pareto_vec]
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    veg_pareto_dt = veg_pareto_dt.append(
        pd.DataFrame(
            # index=(veg_pareto_dt.index.max()+1,),
            columns=veg_pareto_dt.columns,
            data=(out_dt,),
        )
    )

# print(vegetation_dt)
str_pareto_dt = pd.DataFrame(columns=['Wind_Speed','Assembly','Material','vec_len', 'b','loc','scale'])

str_simple_dt = structure_dt[['Wind Speed','Assembly','Material']].drop_duplicates()

for i in range(len(str_simple_dt)):
    samp_dt = str_simple_dt.iloc[[i]]
    j=samp_dt['Wind Speed'].values[0]
    k=samp_dt['Assembly'].values[0]
    l=samp_dt['Material'].values[0]
    # print([j,k,l])
    pareto_dt = structure_dt[(structure_dt['Wind Speed']==j) &(structure_dt['Assembly']==k) & (structure_dt['Material']==l)]
    pareto_vec = pareto_dt['Total Flying Distance  [m]'].values
    pareto_vec = [float(m) for m in pareto_vec]
    n_emb = len(pareto_vec)
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    out_dt =[j,k,l,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    str_pareto_dt = str_pareto_dt.append(
            pd.DataFrame(
                # index=(veg_pareto_dt.index.max()+1,),
                columns=str_pareto_dt.columns,
                data=(out_dt,),
            )
        )

asm_pareto_dt = pd.DataFrame(columns=['Wind_Speed','Cladding','Wall Type','Material','Roof (Sheathing)','vec_len', 'b','loc','scale'])

asm_simple_dt = assembly_dt[['Wind Speed','Cladding','Wall Type','Material','Roof (Sheathing)']].drop_duplicates()

for i in range(len(str_simple_dt)):
    samp_dt = asm_simple_dt.iloc[[i]]
    j=samp_dt['Wind Speed'].values[0]
    k=samp_dt['Cladding'].values[0]
    l=samp_dt['Wall Type'].values[0]
    m=samp_dt['Material'].values[0]
    n=samp_dt['Roof (Sheathing)'].values[0]
    # print([j,k,l])
    pareto_dt = assembly_dt[(assembly_dt['Wind Speed']==j) &(assembly_dt['Cladding']==k) & (assembly_dt['Wall Type']==l)& (assembly_dt['Material']==m)& (assembly_dt['Roof (Sheathing)']==n)]
    pareto_vec = pareto_dt['Total Flying Distance  [m]'].values
    # print(pareto_vec)
    pareto_vec = [float(o) for o in pareto_vec]

    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,l,m,n,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    asm_pareto_dt = asm_pareto_dt.append(
            pd.DataFrame(
                # index=(veg_pareto_dt.index.max()+1,),
                columns=asm_pareto_dt.columns,
                data=(out_dt,),
            )
        )

/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py

In [9]:
with rasterio.open("/work/Stack_Sonoma_2021_classification.tif") as src:
    class_out_image, class_out_transform = rasterio.mask.mask(src, parcel_dataframe.geometry, crop=True)
    class_out_meta = src.meta

class_out_meta.update({"driver": "GTiff",
                 "height": class_out_image.shape[1],
                 "width": class_out_image.shape[2],
                 "transform": class_out_transform})

with rasterio.open("/work/processing_folder/parcel_class.tif", "w", **class_out_meta) as dest:
    dest.write(class_out_image)


with rasterio.open("/work/Stack_Sonoma_2021_biomass.tif") as src:
    biomass_out_image, biomass_out_transform = rasterio.mask.mask(src, parcel_dataframe.geometry, crop=True)
    biomass_out_meta = src.meta

biomass_out_meta.update({"driver": "GTiff",
                 "height": biomass_out_image.shape[1],
                 "width": biomass_out_image.shape[2],
                 "transform": biomass_out_transform})

with rasterio.open("/work/processing_folder/parcel_biomass.tif", "w", **biomass_out_meta) as dest:
    dest.write(biomass_out_image)

In [10]:
# pareto.rvs(size = 1000000,b=veg_pareto_dt.iloc[[0]][['b']],loc=veg_pareto_dt.iloc[[0]][['loc']],scale=veg_pareto_dt.iloc[[0]][['scale']])

In [11]:
from osgeo import gdal, ogr, osr

in_path = '/work/processing_folder/parcel_class.tif'

out_path = '/work/processing_folder/parcel_class.gpkg'

#  get raster datasource
src_ds = gdal.Open( in_path )
#
srcband = src_ds.GetRasterBand(1)
dst_layername = 'Class'
drv = ogr.GetDriverByName("GPKG")
dst_ds = drv.CreateDataSource( out_path )

sp_ref = osr.SpatialReference()
sp_ref.SetFromUserInput('EPSG:4326')

dst_layer = dst_ds.CreateLayer(dst_layername, srs = sp_ref )

fld = ogr.FieldDefn("Class", ogr.OFTInteger)
dst_layer.CreateField(fld)
dst_field = dst_layer.GetLayerDefn().GetFieldIndex("Class")

gdal.Polygonize( srcband, None, dst_layer, dst_field, [], callback=None )

del src_ds
del dst_ds

In [12]:
vegetation_dataframe = geopandas.read_file('/work/processing_folder/parcel_class.gpkg')

In [13]:
vegetation_dataframe[['Class_str']] =  vegetation_dataframe[['Class']].values
d = { 0 : 'Grass' , 4 : 'Grass', 2 : 'Grass', 7 : 'Grass', 5 : 'Shrub', 11 : 'Shrub', 13 : 'Shrub', 14 : 'Shrub',     1 : 'Tree', 3 : 'Tree', 6 : 'Tree', 8 : 'Tree',9 : 'Tree', 10 : 'Tree', 12 : 'Tree',15 : 'Water'}
vegetation_dataframe.Class_str = vegetation_dataframe.Class_str.map(d)

In [14]:
vegetation_dataframe['biomass'] = zonal_stats(vegetation_dataframe, 
                             '/work/processing_folder/parcel_biomass.tif', 
                             stats="mean", # any other stat will also do
                             categorical=False,
                             nodata = np.nan)
vegetation_dataframe['biomass'] = [i['mean'] for i in vegetation_dataframe['biomass']]

In [15]:
vegetation_dataframe['area'] = vegetation_dataframe.to_crs(6414).area/10000
vegetation_dataframe['MG'] = vegetation_dataframe['area']*vegetation_dataframe['biomass']
# vegetation_dataframe['embers'] = 

In [16]:
print(vegetation_dataframe.iloc[[1]])

   Class                                           geometry Class_str  \
1      4  POLYGON ((-122.60764 38.24263, -122.60764 38.2...     Grass   

     biomass      area       MG  
1  13.571429  0.054891  0.74495  


In [17]:
vegetation_dataframe['embers'] = 0
# print(veg_pareto_dt[veg_pareto_dt.Wind_Speed == 'High'].Subs.values)
for i in veg_pareto_dt[veg_pareto_dt.Wind_Speed == 'High'].Subs.values:
    # print(vegetation_dataframe[vegetation_dataframe.Class_str == i])
    # ['embers'] = 
    vegetation_dataframe.loc[vegetation_dataframe.Class_str == i].('embers',vegetation_dataframe[vegetation_dataframe.Class_str == i]['MG']*veg_pareto_dt[(veg_pareto_dt.Subs == i ) &(veg_pareto_dt.Wind_Speed == 'High')].vec_len[0]*1000/5)





SyntaxError: invalid syntax (4145832032.py, line 6)

In [77]:
vegetation_dataframe

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5eceab32-5823-49b2-9c95-72828e51fba3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>